# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Project 4: Web Scraping Job Postings

## Business Case Overview

You're working as a data scientist for a contracting firm that's rapidly expanding. Now that they have their most valuable employee (you!), they need to leverage data to win more contracts. Your firm offers technology and scientific solutions and wants to be competitive in the hiring market. Your principal has two main objectives:

   1. Determine the industry factors that are most important in predicting the salary amounts for these data.
   2. Determine the factors that distinguish job categories and titles from each other. For example, can required skills accurately predict job title?

To limit the scope, your principal has suggested that you *focus on data-related job postings*, e.g. data scientist, data analyst, research scientist, business intelligence, and any others you might think of. You may also want to decrease the scope by *limiting your search to a single region.*

Hint: Aggregators like [Indeed.com](https://www.indeed.com) regularly pool job postings from a variety of markets and industries. 

**Goal:** Scrape your own data from a job aggregation tool like Indeed.com in order to collect the data to best answer these two questions.

---

## Directions

In this project you will be leveraging a variety of skills. The first will be to use the web-scraping and/or API techniques you've learned to collect data on data jobs from Indeed.com or another aggregator. Once you have collected and cleaned the data, you will use it to answer the two questions described above.

### QUESTION 1: Factors that impact salary

To predict salary you will be building either a classification or regression model, using features like the location, title, and summary of the job. If framing this as a regression problem, you will be estimating the listed salary amounts. You may instead choose to frame this as a classification problem, in which case you will create labels from these salaries (high vs. low salary, for example) according to thresholds (such as median salary).

You have learned a variety of new skills and models that may be useful for this problem:
- NLP
- Unsupervised learning and dimensionality reduction techniques (PCA, clustering)
- Ensemble methods and decision tree models
- SVM models

Whatever you decide to use, the most important thing is to justify your choices and interpret your results. *Communication of your process is key.* Note that most listings **DO NOT** come with salary information. You'll need to able to extrapolate or predict the expected salaries for these listings.

### QUESTION 2: Factors that distinguish job category

Using the job postings you scraped for part 1 (or potentially new job postings from a second round of scraping), identify features in the data related to job postings that can distinguish job titles from each other. There are a variety of interesting ways you can frame the target variable, for example:
- What components of a job posting distinguish data scientists from other data jobs?
- What features are important for distinguishing junior vs. senior positions?
- Do the requirements for titles vary significantly with industry (e.g. healthcare vs. government)?

You may end up making multiple classification models to tackle different questions. Be sure to clearly explain your hypotheses and framing, any feature engineering, and what your target variables are. The type of classification model you choose is up to you. Be sure to interpret your results and evaluate your models' performance.


### BONUS PROBLEM

Your boss would rather tell a client incorrectly that they would get a lower salary job than tell a client incorrectly that they would get a high salary job. Adjust one of your models to ease his mind, and explain what it is doing and any tradeoffs. Plot the ROC curve.

---

## Requirements

1. Scrape and prepare your own data.

2. **Create and compare at least two models for each section**. One of the two models should be a decision tree or ensemble model. The other can be a classifier or regression of your choosing (e.g. Ridge, logistic regression, KNN, SVM, etc).
   - Section 1: Job Salary Trends
   - Section 2: Job Category Factors

3. Prepare a polished Jupyter Notebook with your analysis for a peer audience of data scientists. 
   - Make sure to clearly describe and label each section.
   - Comment on your code so that others could, in theory, replicate your work.

4. A brief writeup in an executive summary, written for a non-technical audience.
   - Writeups should be at least 500-1000 words, defining any technical terms, explaining your approach, as well as any risks and limitations.

#### BONUS

5. Answer the salary discussion by using your model to explain the tradeoffs between detecting high vs low salary positions.

6. Convert your executive summary into a public blog post of at least 500 words, in which you document your approach in a tutorial for other aspiring data scientists. Link to this in your notebook.

---

## Suggestions for Getting Started

1. Collect data from [Indeed.com](www.indeed.com) (or another aggregator) on data-related jobs to use in predicting salary trends for your analysis.
  - Select and parse data from *at least 1000 postings* for jobs, potentially from multiple location searches.
2. Find out what factors most directly impact salaries (e.g. title, location, department, etc).
  - Test, validate, and describe your models. What factors predict salary category? How do your models perform?
3. Discover which features have the greatest importance when determining a low vs. high paying job.
  - Your Boss is interested in what overall features hold the greatest significance.
  - HR is interested in which SKILLS and KEY WORDS hold the greatest significance.   
4. Author an executive summary that details the highlights of your analysis for a non-technical audience.
5. If tackling the bonus question, try framing the salary problem as a classification problem detecting low vs. high salary positions.

---

## Useful Resources

- Scraping is one of the most fun, useful and interesting skills out there. Don’t lose out by copying someone else's code!
- [Here is some advice on how to write for a non-technical audience](http://programmers.stackexchange.com/questions/11523/explaining-technical-things-to-non-technical-people)
- [Documentation for BeautifulSoup can be found here](http://www.crummy.com/software/BeautifulSoup/).

---

### Project Feedback + Evaluation

For all projects, students will be evaluated on a simple 3 point scale (0, 1, or 2). Instructors will use this rubric when scoring student performance on each of the core project **requirements:** 

Score | Expectations
----- | ------------
**0** | _Does not meet expectations. Try again._
**1** | _Meets expectations. Good job._
**2** | _Surpasses expectations. Brilliant!_

[For more information on how we grade our DSI projects, see our project grading walkthrough.](https://git.generalassemb.ly/dsi-projects/readme/blob/master/README.md)


In [1]:
import os
import random
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys

## 1. Webscrapping

### 1.1 Save all the job links to csv file

In [2]:
job_links_starting_time = datetime.datetime.now()
print("Scrapping job links is starting at ",job_links_starting_time)

search_word = "data"
links = []
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver')

for page_no in range(0,300):
    #print("Page no {}".format(page_no))
    driver.get("https://www.mycareersfuture.sg/search?search={}&sortBy=new_posting_date&page={}".format(search_word,page_no))
 
    sleep(random.randint(4,8))
    assert "MyCareersFuture" in driver.title
    
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    all_elements = soup.find_all("a",{"class": "bg-white mb3 w-100 dib v-top pa3 no-underline flex-ns flex-wrap JobCard__card___22xP3"})
    for each_entry in all_elements:
        links.append("https://www.mycareersfuture.sg"+each_entry.get('href'))

    next_page_signs = soup.find_all("span",{"type":"action"})
    page_signs = [i.get_text() for i in next_page_signs]
            
    if page_no%10 == 0:
        print("Page {} is completed.....".format(page_no))
        link_df = pd.DataFrame({'urls':links})
        link_df.to_csv("./datasets/MyCareersFuture_Job_Links_Up_to_page_{}.csv".format(page_no),index=False)
       
    if '❯' not in page_signs:
        print("{} is the last page to scrape....".format(page_no))
        break

driver.close()

# Create the dataframe for urls and save it to csv file so that no need to run again.
link_table = pd.DataFrame({'No':range(1,len(links)+1),'urls':links})
link_table.to_csv("./datasets/MyCareersFuture_All_Job_Links.csv",index=False)

# Deleting the backup csv files which are created during scrapping process.
for i in range(0,page_no+10,10):
    if os.path.exists("./datasets/MyCareersFuture_Job_Links_Up_to_page_{}.csv".format(i)):
        os.remove("./datasets/MyCareersFuture_Job_Links_Up_to_page_{}.csv".format(i))
    else:
        print("The file does not exist")

job_links_ending_time = datetime.datetime.now()
print("Scrapping job links is ending at ",job_links_ending_time)
diff = job_links_ending_time - job_links_starting_time
print("Total operation hours are",int(diff.seconds/(60*60)),"hours",
      (diff.seconds%(60*60))/60,"minutes")


Scrapping job links is starting at  2019-05-07 23:25:02.622192
Page 0 is completed.....
Page 10 is completed.....
Page 20 is completed.....
Page 30 is completed.....
Page 40 is completed.....
Page 50 is completed.....
Page 60 is completed.....
Page 70 is completed.....
Page 80 is completed.....
Page 90 is completed.....
Page 100 is completed.....
Page 110 is completed.....
Page 120 is completed.....
Page 130 is completed.....
Page 140 is completed.....
Page 150 is completed.....
Page 160 is completed.....
Page 170 is completed.....
Page 180 is completed.....
Page 190 is completed.....
Page 200 is completed.....
Page 210 is completed.....
Page 220 is completed.....
Page 230 is completed.....
236 is the last page to scrape....
The file does not exist
Scrapping job links is ending at  2019-05-07 23:53:11.531912
Total operation hours are 0 hours 28.133333333333333 minutes


In [3]:
print(len(links))

4736


### 1.2 Inspecting the links that we scrapped

In [4]:
link_df = pd.read_csv("./datasets/MyCareersFuture_All_Job_Links.csv")
print("Before checking duplicates : ",link_df.shape)
link_df.urls.duplicated(keep='first')
print("After checking duplicates : ",link_df.shape)
print(link_df.head())
print(link_df.tail())
print("\nFirst link : ",link_df.urls[0])

Before checking duplicates :  (4736, 2)
After checking duplicates :  (4736, 2)
   No                                               urls
0   1  https://www.mycareersfuture.sg/job/data-scient...
1   2  https://www.mycareersfuture.sg/job/data-center...
2   3  https://www.mycareersfuture.sg/job/data-engine...
3   4  https://www.mycareersfuture.sg/job/field-opera...
4   5  https://www.mycareersfuture.sg/job/senior-data...
        No                                               urls
4731  4732  https://www.mycareersfuture.sg/job/principal-m...
4732  4733  https://www.mycareersfuture.sg/job/deputy-dire...
4733  4734  https://www.mycareersfuture.sg/job/mha-science...
4734  4735  https://www.mycareersfuture.sg/job/business-an...
4735  4736  https://www.mycareersfuture.sg/job/business-an...

First link :  https://www.mycareersfuture.sg/job/data-scientist-panasonic-asia-pacific-a19e1b45f8108c2367d59174ef3b1986


### 1.3 Scrapping each job link to get info

In [5]:
jobs_df = pd.DataFrame(columns=['url','id','title','company','location',
                                'employment_type','seniority','min_experience',
                                'salary_range','salary_type','no_applicants',
                                'posted_date','closing_date','description','requirement'])

In [6]:
def get_soup(url,html,row):
    soup = BeautifulSoup(html,'lxml')

    try:
        jobs_df.loc[row,'url'] = url
    except:
        jobs_df.loc[row,'url'] = "None"
        
    try:
        jobs_df.loc[row,'id'] = soup.find('span',{'class':'black-60 db f6 fw4 mv1'}).get_text()
    except:
        jobs_df.loc[row,'id'] = "None"
    
    try:
        jobs_df.loc[row,'title'] = soup.find('h1',{'id':'job_title'}).get_text()
    except:
        jobs_df.loc[row,'title'] = "None"

    try:
        jobs_df.loc[row,'company'] = soup.find('p',{'name':'company'}).get_text()
    except:
        jobs_df.loc[row,'company'] = "None"

    try:
        jobs_df.loc[row,'location'] = soup.find('p',{'id':'address'}).get_text()
    except:
        jobs_df.loc[row,'location'] = "None"

    try:
        jobs_df.loc[row,'employment_type'] = soup.find('p',{'id':'employment_type'}).get_text()
    except:
        jobs_df.loc[row,'employment_type'] = "None"

    try:
        jobs_df.loc[row,'seniority'] = soup.find('p',{'id':'seniority'}).get_text()
    except:
        jobs_df.loc[row,'seniority'] = "None"

    try:
        jobs_df.loc[row,'min_experience'] = soup.find('p',{'id':'min_experience'}).get_text()
    except:
        jobs_df.loc[row,'min_experience'] = "None"

    try:
        jobs_df.loc[row,'salary_range'] = soup.find('div',{'class':'lh-solid'}).get_text()
    except:
        jobs_df.loc[row,'salary_range'] = "None"
        
    try:
        jobs_df.loc[row,'salary_type'] = soup.find('span',{'class':'salary_type dib f5 fw4 black-60 pr1 i pb'}).get_text()
    except:
        jobs_df.loc[row,'salary_type'] = "None"

    try:
        jobs_df.loc[row,'no_applicants'] = soup.find('span',{'id':'num_of_applications'}).get_text()
    except:
        jobs_df.loc[row,'no_applicants'] = "None"

    try:
        jobs_df.loc[row,'posted_date'] = soup.find('span',{'id':'last_posted_date'}).get_text()
    except:
        jobs_df.loc[row,'posted_date'] = "None"

    try:
        jobs_df.loc[row,'closing_date'] = soup.find('span',{'id':'expiry_date'}).get_text()
    except:
        jobs_df.loc[row,'closing_date'] = "None"

    try:
        jobs_df.loc[row,'description'] = soup.find('div',{'id':'description-content'}).get_text()
    except:
        jobs_df.loc[row,'description'] = "None"  

    try:
        jobs_df.loc[row,'requirement'] = soup.find('div',{'id':'requirements-content'}).get_text()
    except:
        jobs_df.loc[row,'requirement'] = "None"


In [7]:
job_info_starting_time = datetime.datetime.now()
print("Scrapping all jobs info is starting at ",job_info_starting_time)

driver1 = webdriver.Chrome(executable_path='./chromedriver/chromedriver')
driver2 = webdriver.Chrome(executable_path="./chromedriver/chromedriver")
driver3 = webdriver.Chrome(executable_path="./chromedriver/chromedriver")
driver4 = webdriver.Chrome(executable_path="./chromedriver/chromedriver")
driver5 = webdriver.Chrome(executable_path="./chromedriver/chromedriver")
drivers = [driver1,driver2,driver3,driver4,driver5]
#print("Accessing {} .....".format(link_df.urls[0]))
#for i in range(4500,link_df.shape[0],5):   # for testing
for i in range(0,link_df.shape[0]-4,5):
    driver1.get(link_df.urls[i])
    driver2.get(link_df.urls[i+1])
    driver3.get(link_df.urls[i+2])
    driver4.get(link_df.urls[i+3])
    driver5.get(link_df.urls[i+4])

#     print(driver1.title)
#     print(driver2.title)
#     print(driver3.title)
#     print(driver4.title)
#     print(driver5.title)
#     assert "MyCareersFuture" in driver1.title
#     assert "MyCareersFuture" in driver2.title
#     assert "MyCareersFuture" in driver3.title
#     assert "MyCareersFuture" in driver4.title
#     assert "MyCareersFuture" in driver5.title
    
    sleep(random.randint(4,8))
    
    for driver in drivers:
        html = driver.page_source
        get_soup(link_df.urls[i+drivers.index(driver)],html,i+drivers.index(driver))
    
    #if i%10 == 0:   # for testing
    if i%100 == 0:
        print("{} jobs are completed.....".format(i))
        jobs_df.to_csv("./datasets/MyCareersFuture_Scrapping_Up_to_Jobs_{}.csv".format(i),index=False)
    
driver1.close()
driver2.close()
driver3.close()
driver4.close()
driver5.close()

jobs_df.to_csv("./datasets/MyCareersFuture_All_Jobs.csv",index=False)

#for j in range(4500,i+10,10):   # for testing
for j in range(0,i+100,100):
    if os.path.exists("./datasets/MyCareersFuture_Scrapping_Up_to_Jobs_{}.csv".format(j)):
        os.remove("./datasets/MyCareersFuture_Scrapping_Up_to_Jobs_{}.csv".format(j))
    else:
        print("The file does not exist")
        
print("Scrapping Operation is completed successfully.... Total {} jobs are scrapped.".format(i+5))
job_info_ending_time = datetime.datetime.now()
print("Operation is ended at ",job_info_starting_time)
time_diff = job_info_ending_time - job_info_starting_time
print("Total operation hours are",int(time_diff.seconds/(60*60)),"hours",
      int((time_diff.seconds%(60*60))/60),"minutes")


Scrapping all jobs info is starting at  2019-05-07 23:53:11.602539
0 jobs are completed.....
100 jobs are completed.....
200 jobs are completed.....
300 jobs are completed.....
400 jobs are completed.....
500 jobs are completed.....
600 jobs are completed.....
700 jobs are completed.....
800 jobs are completed.....
900 jobs are completed.....
1000 jobs are completed.....
1100 jobs are completed.....
1200 jobs are completed.....
1300 jobs are completed.....
1400 jobs are completed.....
1500 jobs are completed.....
1600 jobs are completed.....
1700 jobs are completed.....
1800 jobs are completed.....
1900 jobs are completed.....
2000 jobs are completed.....
2100 jobs are completed.....
2200 jobs are completed.....
2300 jobs are completed.....
2400 jobs are completed.....
2500 jobs are completed.....
2600 jobs are completed.....
2700 jobs are completed.....
2800 jobs are completed.....
2900 jobs are completed.....
3000 jobs are completed.....
3100 jobs are completed.....
3200 jobs are com